In [14]:
import pandas as pd
from glob import glob
import os
import ipywidgets
import cv2
import matplotlib.pyplot as plt
import ast
import numpy as np

In [11]:

p1_frontal_df = pd.read_csv('../datasets/kor_multi_modal/prep_part1_frontal_face.csv')
p2_frontal_df = pd.read_csv('../datasets/kor_multi_modal/prep_part2_frontal_face.csv')
p3_frontal_df = pd.read_csv('../datasets/kor_multi_modal/prep_part3_frontal_face.csv')
p4_frontal_df = pd.read_csv('../datasets/kor_multi_modal/prep_part4_frontal_face.csv')
p5_frontal_df = pd.read_csv('../datasets/kor_multi_modal/prep_part5_frontal_face.csv')

dataframe = [p1_frontal_df, p2_frontal_df, p3_frontal_df, p4_frontal_df, p5_frontal_df]
p_frontal_df = pd.concat(dataframe)

In [28]:
p_frontal_df = p_frontal_df.reset_index(drop=True)

In [57]:
p_frontal_df.head(2)

,emotion,face_rect,img_path
0,neutral,"{'max_y': 809, 'max_x': 1137, 'min_x': 720, 'm...",part1/KETI_MULTIMODAL_0000000375/KETI_MULTIMOD...
1,neutral,"{'max_y': 809, 'max_x': 1137, 'min_x': 720, 'm...",part1/KETI_MULTIMODAL_0000000375/KETI_MULTIMOD...


In [33]:
def prep_f(img_path):
    return "/".join(img_path.split("/")[-5:])

img_pathes = p_frontal_df['img_path'].map(lambda x : prep_f(x))
# p_frontal_df['img_path'][:100].map(lambda x : prep_f(x))

In [34]:
p_frontal_df['img_path'] = img_pathes

In [38]:
p_frontal_df['img_path'][0]

'part1/KETI_MULTIMODAL_0000000375/KETI_MULTIMODAL_0000000375/KETI_SHOT_0000123017/KM_0000000634.jpg'

In [53]:
BASE_PATH='../../../datasets/emotion/multi-modal'

In [64]:
def f(x):
    img_path = p_frontal_df['img_path'][x]
    img_path = BASE_PATH + "/" + img_path 
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    print(img_path)
    face_rect_dict = ast.literal_eval(p_frontal_df['face_rect'][x])
    max_y = face_rect_dict['max_y']
    max_x = face_rect_dict['max_x']
    min_y = face_rect_dict['min_y']
    min_x = face_rect_dict['min_x']
    cv2.rectangle(img, (min_x, min_y), (max_x, max_y), [0, 255, 0], 3)
    print(p_frontal_df['emotion'][x])
    plt.imshow(img)
    
ipywidgets.interact(f, x = ipywidgets.IntSlider(0, min=0, max=p_frontal_df.shape[0]-1, step=1))

interactive(children=(IntSlider(value=0, description='x', max=1157797), Output()), _dom_classes=('widget-inter…

<function __main__.f(x)>

In [52]:
!ls ../../../datasets/emotion/multi-modal/

download.sh  part1	part2	   part3      part4	 part5
nohup.out    part1.zip	part2.zip  part3.zip  part4.zip  part5.zip


# Check frontal face

## 데이터셋 벨런싱 검증 : OK
## shuffle True 셋팅여부 : OK

In [2]:
frontal_df = pd.read_csv("../datasets/kor_multi_modal/prep_col_all_mdf_frontal_face.csv")

In [32]:
frontal_df.groupby('emotion').count()

,pixels,Usage
emotion,,
0,18357,0
1,18357,0
2,18357,0
3,18357,0
4,18357,0
5,18357,0
6,18357,0


In [34]:
18357 * 7, frontal_df.shape

(128499, (128499, 3))

In [7]:
len_df = frontal_df.shape[0]

In [29]:
emotion_map = {
    0:'anger'
    ,1:'disgust, contempt'
    ,2:'afraid'
    ,3:'happiness'
    ,4:'sadness'
    ,5:'surprise'
    ,6:'neutral'
}

In [50]:
def f(x):
    row = frontal_df.iloc[x]
    pixels = row['pixels']
    emotion = row['emotion']
    img = np.asarray(pixels.split(" "), dtype=np.int16).reshape((48, 48))
    plt.imshow(img, 'gray')
    print(emotion_map[emotion])
    
ipywidgets.interact(f, x = ipywidgets.IntSlider(0, 1, len_df-1, 1))

interactive(children=(IntSlider(value=1, description='x', max=128498, min=1), Output()), _dom_classes=('widget…

<function __main__.f(x)>

## fer데이터와 비교: 거의 유사하거나 kor데이터의 Quality가 더 좋아보임

In [36]:
fer_df = pd.read_csv("../datasets/fer2013/fer2013.csv")

In [38]:
def fer_f(x):
    row = fer_df.iloc[x]
    pixels = row['pixels']
    emotion = row['emotion']
    img = np.asarray(pixels.split(" "), dtype=np.int16).reshape((48, 48))
    plt.imshow(img, 'gray')
    print(emotion_map[emotion])
    
ipywidgets.interact(fer_f, x = ipywidgets.IntSlider(0, 1, len_df-1, 1))

interactive(children=(IntSlider(value=1, description='x', max=128498, min=1), Output()), _dom_classes=('widget…

<function __main__.fer_f(x)>

## Image batch generator 확인
### csv파일 읽어올때 shuffle이 안되고 있었음
    => pandas .read_csv() 에서 sample()으로 shuffle되게 변경

In [42]:
from utils.datasets import DataManager
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# config
dataset_name = 'kor_multi_modal'
input_shape = (64, 64, 1)

# data generator
data_generator = ImageDataGenerator(
                        featurewise_center=False,
                        featurewise_std_normalization=False,
                        rotation_range=10,
                        width_shift_range=0.1,
                        height_shift_range=0.1,
                        zoom_range=.1,
                        horizontal_flip=True)

data_loader = DataManager(dataset_name, image_size=input_shape[:2])
faces, emotions = data_loader.get_data()

/tf/notebooks/face_emotion/face_classification_modi/src/utils/datasets.py:91: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  emotions = pd.get_dummies(data['emotion']).as_matrix()


In [46]:
faces.shape, emotions.shape

((128499, 64, 64, 1), (128499, 7))

In [47]:
suffle_frontal_df = frontal_df.sample(frac=1).reset_index(drop=True)

In [49]:
suffle_frontal_df.head(100)

,emotion,pixels,Usage
0,1,76 76 50 40 41 49 59 82 109 129 139 143 144 14...,NaN
1,3,44 43 48 46 44 39 29 24 24 25 26 30 35 35 37 3...,NaN
2,3,199 200 200 201 201 201 186 71 33 27 22 18 16 ...,NaN
3,3,27 28 24 23 24 26 26 27 24 28 29 27 26 35 42 3...,NaN
4,6,24 29 31 27 28 30 37 39 35 37 55 60 62 52 47 4...,NaN
5,2,163 83 57 57 56 59 46 37 33 34 33 33 26 19 18 ...,NaN
6,4,41 41 41 42 41 40 42 39 40 40 38 41 41 43 43 4...,NaN
7,4,7 7 8 9 9 8 9 9 8 8 8 8 8 8 9 11 12 12 11 13 1...,NaN
8,6,51 58 45 39 31 27 28 27 28 35 42 45 49 50 52 5...,NaN
9,1,41 44 41 39 38 38 38 45 42 42 36 41 37 37 36 3...,NaN


## Attention test

In [95]:
import tensorflow as tf
input_dims = 48*48

In [99]:
# 다음 링크 참조: https://github.com/philipperemy/keras-attention-mechanism
# inputs = Input(shape=(input_dims,))
# attention_probs = Dense(input_dims, activation='softmax', name='attention_probs')(inputs)
# attention_mul = merge([inputs, attention_probs], output_shape=input_dims, name='attention_mul', mode='mul')

def test_x(x):
    row = frontal_df.iloc[x]
    pixels = row['pixels']
    emotion = row['emotion']
    print(emotion_map[emotion])
    img = np.asarray(pixels.split(" "), dtype=np.int16).reshape((48, 48))

    img = img / 255.
    img_tensor = tf.cast(img.reshape(48*48), tf.float32)
    img_tensor = tf.expand_dims(img_tensor, axis=0)

    # Attention model
    inputs = tf.keras.Input(input_dims)
    attention_probs = tf.keras.layers.Dense(input_dims, activation='softmax', name='attention_probs')(inputs)
    attention_mul = tf.keras.layers.multiply([inputs, attention_probs])
    model = tf.keras.models.Model(inputs, attention_mul)
    output_tensor = model(img_tensor)

    print(np.unique(output_tensor.numpy()))

    fig = plt.figure(figsize=(10, 10))
    fig.add_subplot(1, 2, 1)
    plt.imshow(img, 'gray')
    fig.add_subplot(1, 2, 2)
    plt.imshow(tf.reshape(output_tensor, shape=(48,48)).numpy(), 'gray')
    
ipywidgets.interactive(test_x, x = ipywidgets.IntSlider(0, 0, frontal_df.shape[0]-1, 1))

interactive(children=(IntSlider(value=0, description='x', max=128498), Output()), _dom_classes=('widget-intera…

In [125]:
# 다음 논문 참조: https://arxiv.org/pdf/1811.02360.pdf

def test_x(x):
    row = frontal_df.iloc[x]
    pixels = row['pixels']
    emotion = row['emotion']
    print(emotion_map[emotion])
    img = np.asarray(pixels.split(" "), dtype=np.int16).reshape((48, 48))

    img = img / 255.
    img_tensor = tf.cast(img, tf.float32)
    img_tensor = tf.expand_dims(img_tensor, axis=0)

    # Attention model
    inputs = tf.keras.Input((48, 48))
    conv_inputs = tf.expand_dims(inputs, axis=-1)
    conv1 = tf.keras.layers.Conv2D(3, 1, padding='same')(conv_inputs)
    conv3_1 = tf.keras.layers.Conv2D(3, 3, padding='same')(conv_inputs)
    conv3_2 = tf.keras.layers.Conv2D(3, 3, padding='same')(conv3_1)
    mid_add = tf.keras.layers.add([conv3_2, conv1])
    
    ch_concat = tf.keras.layers.concatenate([conv3_1, conv3_2, conv1])
    ch_concat = tf.keras.layers.Conv2D(3, 1, padding='same')(ch_concat)
    ch_concat = tf.keras.backend.mean(ch_concat, axis=-1, keepdims=True)
    
    mid_mul = tf.keras.layers.multiply([mid_add, ch_concat])
    
    res_attention_output = tf.keras.layers.add([mid_add, mid_mul])
    res_attention_output = tf.keras.layers.Conv2D(1, 1, padding='same')(res_attention_output)

    model = tf.keras.models.Model(inputs, res_attention_output)
    output_tensor = model(img_tensor)

    fig = plt.figure(figsize=(10, 10))
    fig.add_subplot(1, 2, 1)
    plt.imshow(img, 'gray')
    fig.add_subplot(1, 2, 2)

    plt.imshow(output_tensor.numpy()[0, :, :, 0], 'gray')
    
ipywidgets.interactive(test_x, x = ipywidgets.IntSlider(0, 0, frontal_df.shape[0]-1, 1))

interactive(children=(IntSlider(value=0, description='x', max=128498), Output()), _dom_classes=('widget-intera…